### 1. Load data

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2)

### 2. Train model & log parameters

In [ ]:
import mlflow
from mlflow.models import infer_signature
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Set the tracking server uri
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Set the experiment name
mlflow.set_experiment("demo 2")

# Test different hyperparameters
hyperparams = [{"C": 0.1}, {"C": 1.0}, {"C": 10.0}]

for params in hyperparams:
    c_value = params["C"]
    with mlflow.start_run(run_name=f"lr-{c_value}"):
        # Log hyperparameters
        mlflow.log_params(params)

        # Train model
        model = LogisticRegression(**params, max_iter=200)
        model.fit(X_train, y_train)

        # Calculate metrics
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')

        # Log metrics
        mlflow.log_metrics({
                "accuracy": round(accuracy, 2), 
                "precision": round(precision, 2), 
                "recall": round(recall, 2)
            })

        # get model signature
        signature = infer_signature(
            model_input=X_train,
            model_output=model.predict(X_train)
        )

        # Log model
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="model",
            signature=signature,
        )

### 3. Register the model

In [ ]:
model_name = "sk-learn-iris-lr"
run_id = input("Enter the Run ID: ")
model_uri = f"runs:/{run_id}/{model_name}"

result = mlflow.register_model(
    model_uri, model_name,
)

### 4. Load the model

In [ ]:
import pandas as pd

run_id = input("Enter the Run ID: ")
model_uri = f"runs:/{run_id}/model"
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Prepare valid input data (Iris example)
input_data = pd.DataFrame({
    "sepal_length": [5.1],
    "sepal_width": [3.5],
    "petal_length": [1.4],
    "petal_width": [0.2]
})

# Predict
predictions = loaded_model.predict(input_data)
print("Predicted class:", predictions)